In [5]:
import cv2
import os

def crop_and_resize_leaf(image_path, output_folder, num_parts=10, overlap=0.2, max_width=600, visualize=False):
    """
    Crop a leaf image into overlapping parts and resize frames for optimized stitching.

    Parameters:
    - image_path: path to the full leaf image
    - output_folder: folder to save cropped frames
    - num_parts: total number of frames to generate
    - overlap: fraction of overlap between consecutive frames (0.2–0.3 recommended)
    - max_width: maximum width to resize frames (keeps aspect ratio)
    - visualize: if True, display each cropped frame for debugging
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load image
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    # Compute height of each cropped frame considering overlap
    frame_height = int(h / (num_parts - (num_parts-1)*overlap))

    for i in range(num_parts):
        # Compute start and end y-coordinate with overlap
        y_start = int(i * frame_height * (1 - overlap))
        y_end = y_start + frame_height
        if y_end > h:
            y_end = h
            y_start = max(0, h - frame_height)

        cropped = img[y_start:y_end, :].copy()

        # Resize for speed while keeping aspect ratio
        current_h, current_w = cropped.shape[:2]
        if current_w > max_width:
            new_h = int(current_h * max_width / current_w)
            cropped = cv2.resize(cropped, (max_width, new_h), interpolation=cv2.INTER_AREA)

        # Save frame
        filename = os.path.join(output_folder, f"frame_{i+1:02d}.png")
        cv2.imwrite(filename, cropped)
        print(f"Saved {filename}")

        # Optional visualization for debugging
        if visualize:
            cv2.imshow("Cropped Frame", cropped)
            cv2.waitKey(200)

    if visualize:
        cv2.destroyAllWindows()

    print(f"Generated {num_parts} frames in '{output_folder}' with {int(overlap*100)}% overlap, resized to max width {max_width}px.")

# --- USAGE ---
image_path = "20211231_123305 (Custom).jpg"  # your full leaf image
output_folder = "leaf_frames"
crop_and_resize_leaf(
    image_path, 
    output_folder, 
    num_parts=6, 
    overlap=0,   # 25% overlap for more robust stitching
    max_width=400, 
    visualize=True  # set False if you don't want preview
)


Saved leaf_frames\frame_01.png
Saved leaf_frames\frame_02.png
Saved leaf_frames\frame_03.png
Saved leaf_frames\frame_04.png
Saved leaf_frames\frame_05.png
Saved leaf_frames\frame_06.png
Generated 6 frames in 'leaf_frames' with 0% overlap, resized to max width 400px.


In [36]:
import cv2
import numpy as np
import glob

# -------------------------------
# ORB + Matcher Setup
# -------------------------------
orb = cv2.ORB_create(1000)  # reduced features for speed
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
MAX_FRAME_WIDTH = 600  # downsize for speed

def preprocess(frame, width=MAX_FRAME_WIDTH):
    """Resize frame to fixed width to speed up ORB"""
    h, w = frame.shape[:2]
    if w > width:
        new_h = int(h * width / w)
        frame = cv2.resize(frame, (width, new_h), interpolation=cv2.INTER_AREA)
    return frame

def linear_blend(target, warped, mask_warped):
    """Simple linear blending in overlap regions"""
    overlap = (mask_warped.astype(np.uint8) & (cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)>0).astype(np.uint8))
    if overlap.sum() == 0:
        target[mask_warped==1] = warped[mask_warped==1]
        return target
    dist = cv2.distanceTransform((overlap==0).astype(np.uint8), cv2.DIST_L2, 5)
    maxd = dist.max() if dist.max()>0 else 1.0
    alpha = np.clip(dist / maxd, 0.0, 1.0)[...,None]
    mask = mask_warped.astype(bool)
    target[mask] = (warped[mask].astype(np.float32) * (1-alpha[mask]) + target[mask].astype(np.float32) * alpha[mask]).astype(np.uint8)
    return target

def stitch_frames(frame_sequence):
    """Stitch frames robustly, with fallback if ORB fails"""
    if len(frame_sequence) == 0:
        raise ValueError("No frames provided for stitching")

    first = preprocess(frame_sequence[0])
    H, W = first.shape[:2]

    # create big canvas
    big_canvas = np.zeros((H*4, W*3, 3), dtype=np.uint8)
    big_mask = np.zeros((H*4, W*3), dtype=np.uint8)
    y0 = 10
    x0 = (big_canvas.shape[1] - W)//2
    big_canvas[y0:y0+H, x0:x0+W] = first
    big_mask[y0:y0+H, x0:x0+W] = 255

    prev_kp, prev_des = orb.detectAndCompute(cv2.cvtColor(first, cv2.COLOR_BGR2GRAY), None)

    for i, raw in enumerate(frame_sequence[1:], start=1):
        frame = preprocess(raw)
        gframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, des = orb.detectAndCompute(gframe, None)
        if des is None or prev_des is None:
            print(f"Frame {i}: ORB failed, using fallback stacking")
            # fallback stacking
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        matches = bf.match(des, prev_des)
        matches = sorted(matches, key=lambda x: x.distance)[:100]
        print(f"Frame {i}: {len(kp)} keypoints, {len(matches)} matches")

        if len(matches) < 8:
            print(f"Frame {i}: Not enough matches, using fallback stacking")
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        # Compute homography
        src_pts = np.float32([ kp[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ prev_kp[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
        Hmat, maskH = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if Hmat is None:
            print(f"Frame {i}: Homography failed, using fallback stacking")
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        # Warp and blend
        warped = cv2.warpPerspective(frame, Hmat, (big_canvas.shape[1], big_canvas.shape[0]))
        warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped_mask = (warped_gray > 10).astype(np.uint8)
        big_canvas = linear_blend(big_canvas, warped, warped_mask)
        big_mask = np.clip(big_mask + warped_mask*255, 0, 255)
        prev_kp, prev_des = kp, des

    # Crop non-empty region
    ys, xs = np.where(big_mask>0)
    if ys.size == 0:
        return big_canvas
    miny, maxy = ys.min(), ys.max()
    minx, maxx = xs.min(), xs.max()
    result = big_canvas[miny:maxy+1, minx:maxx+1]

    return result

# 1. Load your pre-cropped/partial leaf images
frame_files = sorted(glob.glob("leaf_frames/frame_*.png"))  # your cropped frames folder
frames = [cv2.imread(f) for f in frame_files]

# 2. Stitch frames
stitched_leaf = stitch_frames(frames)

# 3. Save and display result
cv2.imwrite("stitched_leaf_test.png", stitched_leaf)
cv2.imshow("Stitched Leaf", stitched_leaf)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("✅ Stitched leaf saved as stitched_leaf_test.png")


Frame 1: ORB failed, using fallback stacking
Frame 2: 2 keypoints, 1 matches
Frame 2: Not enough matches, using fallback stacking
Frame 3: 3 keypoints, 2 matches
Frame 3: Not enough matches, using fallback stacking
Frame 4: ORB failed, using fallback stacking
Frame 5: ORB failed, using fallback stacking
✅ Stitched leaf saved as stitched_leaf_test.png


In [37]:
import cv2
import numpy as np
import glob
import os

# -------------------------------
# --- Linear blending ---
# -------------------------------
def linear_blend(target, warped, mask_warped):
    overlap = (mask_warped.astype(np.uint8) & (cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)>0).astype(np.uint8))
    if overlap.sum() == 0:
        target[mask_warped==1] = warped[mask_warped==1]
        return target
    dist = cv2.distanceTransform((overlap==0).astype(np.uint8), cv2.DIST_L2, 5)
    maxd = dist.max() if dist.max()>0 else 1.0
    alpha = np.clip(dist / maxd, 0.0, 1.0)[...,None]
    mask = mask_warped.astype(bool)
    target[mask] = (warped[mask].astype(np.float32) * (1-alpha[mask]) + target[mask].astype(np.float32) * alpha[mask]).astype(np.uint8)
    return target

# -------------------------------
# --- Robust stitching ---
# -------------------------------
def stitch_frames(frames):
    if len(frames) == 0:
        raise ValueError("No frames provided")

    first = frames[0]
    H, W = first.shape[:2]
    big_canvas = np.zeros((H*4, W*3, 3), dtype=np.uint8)
    big_mask = np.zeros((H*4, W*3), dtype=np.uint8)
    y0 = 10
    x0 = (big_canvas.shape[1] - W)//2
    big_canvas[y0:y0+H, x0:x0+W] = first
    big_mask[y0:y0+H, x0:x0+W] = 255

    detector = cv2.AKAZE_create()
    prev_kp, prev_des = detector.detectAndCompute(cv2.cvtColor(first, cv2.COLOR_BGR2GRAY), None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    for frame in frames[1:]:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, des = detector.detectAndCompute(gray, None)
        if des is None or prev_des is None:
            # fallback stacking
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        matches = bf.match(des, prev_des)
        matches = sorted(matches, key=lambda x: x.distance)[:150]

        if len(matches) < 8:
            # fallback stacking
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        src_pts = np.float32([ kp[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ prev_kp[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
        Hmat, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        if Hmat is None:
            # fallback stacking
            ys = np.where(big_mask.any(axis=1))[0]
            bottom = ys.max() if ys.size else 0
            yplace = min(bottom + 5, big_canvas.shape[0] - frame.shape[0])
            xplace = (big_canvas.shape[1] - frame.shape[1]) // 2
            big_canvas[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = frame
            big_mask[yplace:yplace+frame.shape[0], xplace:xplace+frame.shape[1]] = 255
            prev_kp, prev_des = kp, des
            continue

        warped = cv2.warpPerspective(frame, Hmat, (big_canvas.shape[1], big_canvas.shape[0]))
        warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped_mask = (warped_gray > 10).astype(np.uint8)
        big_canvas = linear_blend(big_canvas, warped, warped_mask)
        big_mask = np.clip(big_mask + warped_mask*255, 0, 255)
        prev_kp, prev_des = kp, des

    ys, xs = np.where(big_mask>0)
    if ys.size == 0:
        return big_canvas
    return big_canvas[ys.min():ys.max()+1, xs.min():xs.max()+1]

# -------------------------------
# --- Main script ---
# -------------------------------
if __name__ == "__main__":
    # Load all cropped frames (replace folder path)
    frame_files = sorted(glob.glob("leaf_frames/frame_*.png"))
    frames = [cv2.imread(f) for f in frame_files]

    stitched_leaf = stitch_frames(frames)
    cv2.imwrite("stitched_leaf_result.png", stitched_leaf)
    print("✅ Stitched leaf saved as 'stitched_leaf_result.png'.")

    # Optional preview
    cv2.imshow("Stitched Leaf", stitched_leaf)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


✅ Stitched leaf saved as 'stitched_leaf_result.png'.


In [7]:
import cv2
import glob
import numpy as np
import os

# -------------------------------
def estimate_vertical_shift(imgA, imgB):
    colA = imgA.mean(axis=1).astype(np.float32)
    colB = imgB.mean(axis=1).astype(np.float32)
    # pad the shorter array to match length
    if len(colA) != len(colB):
        min_len = min(len(colA), len(colB))
        colA = colA[:min_len]
        colB = colB[:min_len]

    fA = np.fft.fft(colA)
    fB = np.fft.fft(colB)
    R = (fA * np.conj(fB)) / (np.abs(fA * np.conj(fB)) + 1e-9)
    r = np.fft.ifft(R)
    shift = np.argmax(np.abs(r))
    if shift > len(colA)//2:
        shift -= len(colA)
    return int(shift)

def fast_stack(frames):
    canvas = frames[0].copy()
    for i in range(1, len(frames)):
        A = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        B = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        # resize B to match canvas width
        if frames[i].shape[1] != canvas.shape[1]:
            scale = canvas.shape[1] / frames[i].shape[1]
            new_h = int(frames[i].shape[0] * scale)
            B = cv2.resize(B, (canvas.shape[1], new_h))
            frames[i] = cv2.resize(frames[i], (canvas.shape[1], new_h))
        shift = estimate_vertical_shift(A, B)
        if shift > 0:
            crop = frames[i][shift:, :]
            canvas = np.vstack([canvas, crop])
        else:
            canvas = np.vstack([canvas, frames[i]])
    return canvas

# -------------------------------
if __name__ == "__main__":
    frame_files = sorted(glob.glob("leaf_frames/frame_*.png"))
    frames = [cv2.imread(f) for f in frame_files]

    stitched_leaf = fast_stack(frames)
    cv2.imwrite("stitched_leaf_fixed.png", stitched_leaf)
    print("✅ Stitched leaf saved as 'stitched_leaf_fixed.png'.")

    cv2.imshow("Stitched Leaf", stitched_leaf)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


✅ Stitched leaf saved as 'stitched_leaf_fixed.png'.


In [17]:
import cv2
import numpy as np
import os

# -------------------------------
def estimate_vertical_shift(imgA, imgB):
    colA = imgA.mean(axis=1).astype(np.float32)
    colB = imgB.mean(axis=1).astype(np.float32)
    min_len = min(len(colA), len(colB))
    colA = colA[:min_len]
    colB = colB[:min_len]

    fA = np.fft.fft(colA)
    fB = np.fft.fft(colB)
    R = (fA * np.conj(fB)) / (np.abs(fA * np.conj(fB)) + 1e-9)
    r = np.fft.ifft(R)
    shift = np.argmax(np.abs(r))
    if shift > len(colA)//2:
        shift -= len(colA)
    return int(shift)

def fast_stack(frames):
    canvas = frames[0].copy()
    for i in range(1, len(frames)):
        A = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        B = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)

        # resize to match width
        if frames[i].shape[1] != canvas.shape[1]:
            scale = canvas.shape[1] / frames[i].shape[1]
            new_h = int(frames[i].shape[0] * scale)
            B = cv2.resize(B, (canvas.shape[1], new_h))
            frames[i] = cv2.resize(frames[i], (canvas.shape[1], new_h))

        shift = estimate_vertical_shift(A, B)
        if shift > 0:
            crop = frames[i][shift:, :]
            canvas = np.vstack([canvas, crop])
        else:
            canvas = np.vstack([canvas, frames[i]])
    return canvas

# -------------------------------
def simulate_scan_from_video(video_path, frame_skip=10, max_frames=10):
    """
    Simulate scanning a leaf from a video.
    - frame_skip: how many frames to skip (controls step size)
    - max_frames: how many frames to grab
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret or len(frames) >= max_frames:
            break

        if frame_count % frame_skip == 0:
            # resize for speed
            frame = cv2.resize(frame, (600, int(frame.shape[0]*600/frame.shape[1])))
            frames.append(frame)
            print(f"Captured frame {frame_count}")

        frame_count += 1

    cap.release()
    return frames

# -------------------------------
if __name__ == "__main__":
    video_path = "test_scan2.mp4"  # put your test video here
    frames = simulate_scan_from_video(video_path, frame_skip=50, max_frames=10)

    if len(frames) < 2:
        print("⚠️ Not enough frames extracted from video.")
    else:
        stitched = fast_stack(frames)
        cv2.imwrite("stitched_from_video.png", stitched)
        print("✅ Saved stitched leaf as stitched_from_video.png")

        cv2.imshow("Stitched Leaf", stitched)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


Captured frame 0
Captured frame 50
Captured frame 100
✅ Saved stitched leaf as stitched_from_video.png


In [19]:
import cv2
import numpy as np

# -------------------------------
# ORB + BFMatcher setup
orb = cv2.ORB_create(800)  # features per frame
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
MAX_FRAME_WIDTH = 600

def preprocess(frame, width=MAX_FRAME_WIDTH):
    h, w = frame.shape[:2]
    if w > width:
        new_h = int(h * width / w)
        frame = cv2.resize(frame, (width, new_h), interpolation=cv2.INTER_AREA)
    return frame

def linear_blend(target, warped, mask_warped):
    overlap = (mask_warped.astype(np.uint8) & (cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)>0).astype(np.uint8))
    if overlap.sum() == 0:
        target[mask_warped==1] = warped[mask_warped==1]
        return target
    dist = cv2.distanceTransform((overlap==0).astype(np.uint8), cv2.DIST_L2, 5)
    maxd = dist.max() if dist.max()>0 else 1.0
    alpha = np.clip(dist / maxd, 0.0, 1.0)[...,None]
    mask = mask_warped.astype(bool)
    target[mask] = (warped[mask].astype(np.float32) * (1-alpha[mask]) + target[mask].astype(np.float32) * alpha[mask]).astype(np.uint8)
    return target

def stitch_frames(frame_sequence):
    first = preprocess(frame_sequence[0])
    H, W = first.shape[:2]
    big_canvas = np.zeros((H*4, W*3, 3), dtype=np.uint8)
    big_mask = np.zeros((H*4, W*3), dtype=np.uint8)

    y0 = 50
    x0 = (big_canvas.shape[1] - W)//2
    big_canvas[y0:y0+H, x0:x0+W] = first
    big_mask[y0:y0+H, x0:x0+W] = 255

    prev_kp, prev_des = orb.detectAndCompute(cv2.cvtColor(first, cv2.COLOR_BGR2GRAY), None)

    for i, raw in enumerate(frame_sequence[1:], start=2):
        frame = preprocess(raw)
        gframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, des = orb.detectAndCompute(gframe, None)
        if des is None or prev_des is None:
            print(f"Frame {i}: ORB failed, skipping")
            continue

        matches = bf.match(des, prev_des)
        matches = sorted(matches, key=lambda x: x.distance)[:100]

        if len(matches) < 8:
            print(f"Frame {i}: Not enough matches")
            continue

        src_pts = np.float32([ kp[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ prev_kp[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
        Hmat, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if Hmat is None:
            print(f"Frame {i}: Homography failed")
            continue

        warped = cv2.warpPerspective(frame, Hmat, (big_canvas.shape[1], big_canvas.shape[0]))
        warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped_mask = (warped_gray > 10).astype(np.uint8)

        big_canvas = linear_blend(big_canvas, warped, warped_mask)
        big_mask = np.clip(big_mask + warped_mask*255, 0, 255)

        prev_kp, prev_des = kp, des
        print(f"Frame {i}: stitched")

    ys, xs = np.where(big_mask>0)
    if ys.size == 0:
        return big_canvas
    miny, maxy = ys.min(), ys.max()
    minx, maxx = xs.min(), xs.max()
    return big_canvas[miny:maxy+1, minx:maxx+1]

# -------------------------------
def simulate_orb_stitch_from_video(video_path, frame_skip=15, max_frames=8):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret or len(frames) >= max_frames:
            break

        if frame_count % frame_skip == 0:
            frame = cv2.resize(frame, (600, int(frame.shape[0]*600/frame.shape[1])))
            frames.append(frame)
            print(f"Captured frame {len(frames)}")

        frame_count += 1

    cap.release()

    if len(frames) < 2:
        print("⚠️ Not enough frames for stitching")
        return None

    stitched = stitch_frames(frames)
    cv2.imwrite("stitched_orb_video.png", stitched)
    print("✅ Saved stitched leaf as stitched_orb_video.png")

    cv2.imshow("Stitched ORB", stitched)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# -------------------------------
if __name__ == "__main__":
    simulate_orb_stitch_from_video("test_scan2.mp4", frame_skip=10, max_frames=20)


Captured frame 1
Captured frame 2
Captured frame 3
Captured frame 4
Captured frame 5
Captured frame 6
Captured frame 7
Captured frame 8
Captured frame 9
Captured frame 10
Captured frame 11
Captured frame 12
Captured frame 13
Captured frame 14
Captured frame 15
Frame 2: Not enough matches
Frame 3: Not enough matches
Frame 4: Not enough matches
Frame 5: ORB failed, skipping
Frame 6: ORB failed, skipping
Frame 7: Not enough matches
Frame 8: Not enough matches
Frame 9: Not enough matches
Frame 10: Not enough matches
Frame 11: Not enough matches
Frame 12: Not enough matches
Frame 13: ORB failed, skipping
Frame 14: ORB failed, skipping
Frame 15: ORB failed, skipping
✅ Saved stitched leaf as stitched_orb_video.png
